In [ ]:
# Adapted from: https://github.com/Shubha23/Fake-News-Detection-Text-Preprocessing-and-Classification/blob/master/fake-news-detection-text-pre-processing-using-nltk.ipynb
# and: https://github.com/manthan89-py/Fake_News_detection/blob/master/Fake%20News.ipynb

# Import necessary Python libraries, modules, etc. 
import re # for regular expressions
import string as st # for removing punctuation
import numpy as np # for linear algebra
import pandas as pd # for frame processing
import matplotlib.pyplot as plt # for data visualization
import nltk # for natural language processing
from nltk.corpus import stopwords # for removing english stopwords
from nltk.stem import WordNetLemmatizer # for term stemming
import sklearn # for predictive data analysis
from sklearn import preprocessing # for data preprocessing
from sklearn.model_selection import train_test_split # for splitting data into test/train sets
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell # to modify Jupyter notebook configuration
InteractiveShell.ast_node_interactivity = "all" # so that all outputs in a cell are returned (instead of last instance)

In [ ]:
# Load the fake and true news CSVs into Pandas dataframes
true_news = pd.read_csv('True.csv') 
fake_news = pd.read_csv('Fake.csv')

# Add column for fake/true label
true_news['label'] = 'true'
fake_news['label'] = 'fake'

# Preview first 5 rows in datasets to ensure they imported properly
true_news.head()
fake_news.head()

In [ ]:
# Combine true_news and fake_news data frames into one
dfs = [true_news, fake_news]
news_data = pd.concat(dfs)

# Initialize instance of LabelEncoder
le = preprocessing.LabelEncoder()

# Assign numerical values to column of target values (true = 1, fake = 0)
news_data['target'] = le.fit_transform(news_data['label'])

# Concatenate text columns and isolate only relevant columns for analysis (i.e., text and target)
news_data['text'] = news_data['title'] + news_data['text']
news_data = news_data[['text', 'target']]

# Check that binary values were assigned correctly
news_data.shape # data frame dimensions
news_data.head(-5) # first 5 and last 5 rows
news_data.isnull().sum() # check for null values

In [ ]:
# Initialize instance of WordNetLemmatizer() -- reduce words to their roots
wnl = WordNetLemmatizer()

# Download multilingual Wordnet data from OMW
nltk.download('omw-1.4')

# List of english stopwords
nltk.download('stopwords') 
stop_words = set(stopwords.words('english'))

# Download english dictionary ('wordnet')
nltk.download('wordnet')

In [ ]:
# Define function for cleaning data
def data_cleaning(row):
    row = row.lower() # convert text into lowercase
    row = re.sub('[^a-zA-Z]', ' ', row) # remove number and special characters using regex (keep words only)
    token = row.split() # split the data and tokenize it
    news = [wnl.lemmatize(word) for word in token if not word in stop_words] # lemmatize the words and remove any stopwords (e.g., a, an, the, etc.)
    cleaned_news = ' '.join(news) # join all tokenized words with space in between 
    
    return cleaned_news

# Clean the data - might take a couple minutes to run.
news_data['text'] = news_data['text'].apply(lambda x : data_cleaning(x)) # 'text' column

# Check for null values
news_data.isnull().sum()

In [ ]:
# Separate news_data into predictor and response variables
X = news_data.iloc[:40000, 0] # features used to predict if news is fake or true
y = news_data.iloc[:40000, 1] # what we're trying to predict: (whether is fake(0) or true(1))

# Split the data into training and test subsets
train_data, test_data, train_target, test_target = train_test_split(X, y, random_state = 0, train_size = 0.80)

# View first 5 rows to ensure data split worked correctly
train_data.head(-5)
test_data.head(-5)
train_target.head(-5)

In [ ]:
# Initialize vectorizer
vectorizer = TfidfVectorizer(max_features = 50000, lowercase = False, ngram_range = (1,2))

# Vectorize training data
vec_train_data = vectorizer.fit_transform(train_data)
vec_train_data = vec_train_data.toarray()

# Vectorize test data
vec_test_data = vectorizer.transform(test_data).toarray()

# Double check dimensions
train_data.shape , test_data.shape # pre-vectorization
vec_train_data.shape , vec_test_data.shape #post-vectorization

# Double check data partitioning after split
train_target.value_counts() # balanced partition of train data
test_target.value_counts() # balanced partition of test data

In [18]:
# Store vectorized training and test data into respective dfs
training_data = pd.DataFrame(vec_train_data , columns=vectorizer.get_feature_names())
testing_data = pd.DataFrame(vec_test_data , columns= vectorizer.get_feature_names())

,aa,aadhaar,aapl,aaron,aarp,ab,aba,ababa,aback,abadi,...,zoo,zoomph,zor,zor city,zor province,zucker,zuckerberg,zuckerberg said,zuma,zuma said
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,aa,aadhaar,aapl,aaron,aarp,ab,aba,ababa,aback,abadi,...,zoo,zoomph,zor,zor city,zor province,zucker,zuckerberg,zuckerberg said,zuma,zuma said
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
